In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd

from datetime import datetime

from complexity_regularized_dcca.data.synthetic import SyntheticData

from complexity_regularized_dcca.experiments.synthetic import (SynthDeepCCAExperiment,
                                                               SynthDeepCCAEExperiment,
                                                               SynthDeepCCAEChannelwiseExperiment,
                                                               SynthDeepCCAEChannelwisePaperExperiment)

from complexity_regularized_dcca.data.synthetic import SyntheticData
from complexity_regularized_dcca.algorithms.evaluation import SyntheticEvaluation
from complexity_regularized_dcca.algorithms.correlation import CCA
from complexity_regularized_dcca.algorithms.losses_metrics import get_similarity_metric_v1

In [ ]:
# Initialize parameters for the run
np.random.seed(500)
num_runs = 5
rho_vec = np.array([.9,.7,.5,.3])
shared_dim = 2
num_channels=5
N = 400 
num_epochs = 2000
conv_threshold = 1e-6
num_nodes_layer = 32
num_nodes_layer_chw = 48
lambda_l2=1e-4
lambda_rec = 1e-6
lambda_rad_dccae = 1e-2
lambda_rad_idccae = 1e-2
topk_proposed = 1

cca_ccor_rho = np.empty([len(rho_vec),shared_dim])
cca_ccor_rho_trials = np.empty([len(rho_vec),num_runs,shared_dim])
dcca_ccor_rho = np.empty([len(rho_vec),shared_dim])
dcca_ccor_rho_trials = np.empty([len(rho_vec),num_runs,shared_dim])
dccae_ccor_rho = np.empty([len(rho_vec),shared_dim])
dccae_ccor_rho_trials = np.empty([len(rho_vec),num_runs,shared_dim])
non_mult_ccor_rho = np.empty([len(rho_vec),shared_dim])
non_mult_ccor_rho_trials = np.empty([len(rho_vec),num_runs,shared_dim])
proposed_ccor_rho = np.empty([len(rho_vec),shared_dim])
proposed_ccor_rho_trials = np.empty([len(rho_vec),num_runs,shared_dim])
proposed_dccae_ccor_rho = np.empty([len(rho_vec),shared_dim])
proposed_dccae_ccor_rho_trials = np.empty([len(rho_vec),num_runs,shared_dim])

cca_sub_sim_rho = np.empty([len(rho_vec),2])
cca_sub_sim_rho_trials = np.empty([len(rho_vec),num_runs,2])
dcca_sub_sim_rho = np.empty([len(rho_vec),2])
dcca_sub_sim_rho_trials = np.empty([len(rho_vec),num_runs,2])
dccae_sub_sim_rho = np.empty([len(rho_vec),2])
dccae_sub_sim_rho_trials = np.empty([len(rho_vec),num_runs,2])
non_mult_sub_sim_rho = np.empty([len(rho_vec),2])
non_mult_sub_sim_rho_trials = np.empty([len(rho_vec),num_runs,2])
proposed_sub_sim_rho = np.empty([len(rho_vec),2])
proposed_sub_sim_rho_trials = np.empty([len(rho_vec),num_runs,2])
proposed_dccae_sub_sim_rho = np.empty([len(rho_vec),2])
proposed_dccae_sub_sim_rho_trials = np.empty([len(rho_vec),num_runs,2])

# Synthetic Data

In [ ]:
for r,rho in enumerate(rho_vec):
    corr_coeff_deviation = 0.01
    rho_values = np.random.uniform(rho-corr_coeff_deviation,rho+corr_coeff_deviation,shared_dim)
    can_corr_vec = np.concatenate((np.ones(shared_dim)*rho,np.zeros(num_channels-shared_dim)))
    cca_ccor_runs = np.empty([num_runs,shared_dim])
    dcca_ccor_runs = np.empty([num_runs,shared_dim])
    dccae_ccor_runs = np.empty([num_runs,shared_dim])
    non_mult_ccor_runs = np.empty([num_runs,shared_dim])
    proposed_ccor_runs = np.empty([num_runs,shared_dim])
    proposed_dccae_ccor_runs = np.empty([num_runs,shared_dim])
    cca_sub_sim_runs = np.empty([num_runs,2])
    dcca_sub_sim_runs = np.empty([num_runs,2])
    dccae_sub_sim_runs = np.empty([num_runs,2])
    non_mult_sub_sim_runs = np.empty([num_runs,2])
    proposed_sub_sim_runs = np.empty([num_runs,2])
    proposed_dccae_sub_sim_runs = np.empty([num_runs,2])
    
    # generate and save the data
    # Load synthetic data
    syn_dataprovider = SyntheticData.generate(
    num_samples=N,
    batch_size=N,
    correlations=can_corr_vec,
    num_channels=num_channels,
    non_lin_type='channel_wise'
    )

    # Create root dir for all following experiments
    root_dir = '/'+datetime.now().strftime("%d_%m_%Y_%H_%M_%S_%f")
    os.mkdir(root_dir)

    # Save dataset
    syn_dataprovider.save(root_dir)
        
    for run in range(num_runs):        
        #CCA
        dataset = syn_dataprovider.get_dataset()
        for data in dataset:
            pass
        
        _, _, epsilon, omega, ccor, _, _ = CCA(
                data['nn_input_0'],
                data['nn_input_1'],
                shared_dim,
                1e-4,
                1e-4
        )
        cca_sim_v0 = get_similarity_metric_v1(
                epsilon[:shared_dim],
                tf.transpose(data['z_0'])[:shared_dim],
                shared_dim
         )
        cca_sim_v1 = get_similarity_metric_v1(
                omega[:shared_dim],
                tf.transpose(data['z_1'])[:shared_dim],
                shared_dim
         )

        cca_ccor_runs[run] = ccor[:shared_dim]
        cca_sub_sim_runs[run] = [cca_sim_v0,cca_sim_v1]

        print('rho = ',rho)
        print('CCA = ',np.mean([cca_sim_v0,cca_sim_v1]))

        #DCCA
        exp = SynthDeepCCAExperiment(
            log_dir=os.path.join(root_dir, 'dcca'), 
            encoder_config_v1=[(num_nodes_layer, 'relu'),(num_nodes_layer, 'relu'), (shared_dim, None)],
            encoder_config_v2=[(num_nodes_layer, 'relu'),(num_nodes_layer, 'relu'), (shared_dim, None)],
            dataprovider=syn_dataprovider,
            shared_dim=shared_dim,
            lambda_rad=0,
            topk=1,
            max_perc=1,
            lambda_l1=0,
            lambda_l2=lambda_l2,
            cca_reg=1e-4,
            eval_epochs=50,
            val_default_value=1.0,
            convergence_threshold=conv_threshold
        )

        exp.train_multiple_epochs(num_epochs) #
        exp.save()
        
        
         #DCCAE
        exp = SynthDeepCCAEExperiment(
                log_dir=os.path.join(root_dir, 'dccae'), 
                encoder_config_v1=[(num_nodes_layer, 'relu'),(num_nodes_layer, 'relu'), (shared_dim, None)],
                encoder_config_v2=[(num_nodes_layer, 'relu'),(num_nodes_layer, 'relu'), (shared_dim, None)],
                decoder_config_v1=[(num_nodes_layer, 'relu'),(num_nodes_layer, 'relu'), (num_channels, None)],
                decoder_config_v2=[(num_nodes_layer, 'relu'),(num_nodes_layer, 'relu'), (num_channels, None)],
                dataprovider=syn_dataprovider,
                shared_dim=shared_dim,
                lambda_rad=0,
                topk=1,
                max_perc=1,
                lambda_l1=0,
                lambda_l2=1e-4,
                lambda_rec=lambda_rec,
                cca_reg=1e-4,
                eval_epochs=50,
                val_default_value=1.0,
                convergence_threshold=conv_threshold
        )

        exp.train_multiple_epochs(num_epochs) #
        exp.save()
        
        
        
        # IDCCAE Channelwise
        exp = SynthDeepCCAEChannelwiseExperiment(
            log_dir=os.path.join(root_dir, 'dccae_nma_cw'),
            encoder_config=[(num_nodes_layer_chw, 'relu'), (1, None)],
            decoder_config=[(num_nodes_layer_chw, 'relu'), (1, None)],
            dataprovider=syn_dataprovider,
            shared_dim=shared_dim,
            lambda_rad=0,
            topk=1,
            max_perc=1,
            lambda_l1=0,
            lambda_l2=0,
            cca_reg=1e-4,
            eval_epochs=50,
            val_default_value=1.0,
            convergence_threshold=conv_threshold
        )

        exp.train_multiple_epochs(num_epochs)
        exp.save()
       
        # Proposed Channelwise
        exp = SynthDeepCCAEChannelwiseExperiment(
                log_dir=os.path.join(root_dir, 'dccae_proposed_cw'),
                encoder_config=[(num_nodes_layer_chw, 'relu'), (1, None)],
                decoder_config=[(num_nodes_layer_chw, 'relu'), (1, None)],
                dataprovider=syn_dataprovider,
                shared_dim=shared_dim,
                lambda_rad=lambda_rad_idccae,
                topk=topk_proposed,
                max_perc=1,
                lambda_l1=0,
                lambda_l2=0,
                cca_reg=1e-4,
                eval_epochs=50,
                val_default_value=1.0,
                convergence_threshold=conv_threshold
        )

        exp.train_multiple_epochs(num_epochs)
        exp.save()

        #CORE DCCAE
        exp = SynthDeepCCAEExperiment(
                log_dir=os.path.join(root_dir, 'dccae_proposed_ncw'), 
                encoder_config_v1=[(num_nodes_layer, 'relu'),(num_nodes_layer, 'relu'), (shared_dim, None)],
                encoder_config_v2=[(num_nodes_layer, 'relu'),(num_nodes_layer, 'relu'), (shared_dim, None)],
                decoder_config_v1=[(num_nodes_layer, 'relu'),(num_nodes_layer, 'relu'), (num_channels, None)],
                decoder_config_v2=[(num_nodes_layer, 'relu'),(num_nodes_layer, 'relu'), (num_channels, None)],
                dataprovider=syn_dataprovider,
                shared_dim=shared_dim,
                lambda_rad=lambda_rad_dccae,
                topk=1,
                max_perc=1,
                lambda_l1=0,
                lambda_l2=1e-4,
                lambda_rec=lambda_rec,
                cca_reg=1e-4,
                eval_epochs=50,
                val_default_value=1.0,
                convergence_threshold=conv_threshold
        )

        exp.train_multiple_epochs(num_epochs)
        exp.save()
        
        # perform evaluation         
        syn_dataprovider = SyntheticData.from_saved_data(root_dir)
        validator = SyntheticEvaluation(syn_dataprovider, root_dir) 
        results = validator.validate_all(weights_to_load='latest') 
        validator.generate_result_table()
        
        key, value = list(results.items())[0]
        technique = results[key]
        metric = technique[0]
        dcca_ccor_runs[run] = np.array(metric['ccor'])[:shared_dim]
        dcca_sub_sim_runs[run] = [np.array(metric['sim_v0']),np.array(metric['sim_v1'])]   
        
        key, value = list(results.items())[1]
        technique = results[key]
        metric = technique[0]
        dccae_ccor_runs[run] = np.array(metric['ccor'])[:shared_dim]
        dccae_sub_sim_runs[run] = [np.array(metric['sim_v0']),np.array(metric['sim_v1'])]
        
        key, value = list(results.items())[2]
        technique = results[key]
        metric = technique[0]
        ccor_non_mult = np.array(metric['ccor'])
        non_mult_ccor_runs[run] = ccor_non_mult[:shared_dim]
        non_mult_sub_sim_runs[run] = [np.array(metric['sim_v0']),np.array(metric['sim_v1'])]
        
        key, value = list(results.items())[3]
        technique = results[key]
        metric = technique[0]
        ccor_proposed = np.array(metric['ccor'])
        proposed_ccor_runs[run] = ccor_proposed[:shared_dim]
        proposed_sub_sim_runs[run] = [np.array(metric['sim_v0']),np.array(metric['sim_v1'])]
        
        key, value = list(results.items())[4]
        technique = results[key]
        metric = technique[0]
        ccor_proposed_dccae = np.array(metric['ccor'])
        proposed_dccae_ccor_runs[run] = ccor_proposed_dccae[:shared_dim]
        proposed_dccae_sub_sim_runs[run] = [np.array(metric['sim_v0']),np.array(metric['sim_v1'])]

    cca_ccor_rho[r] = np.mean(cca_ccor_runs,axis=0)
    cca_ccor_rho_trials[r] = cca_ccor_runs    
    dcca_ccor_rho[r] = np.mean(dcca_ccor_runs,axis=0)
    dcca_ccor_rho_trials[r] = dcca_ccor_runs
    dccae_ccor_rho[r] = np.mean(dccae_ccor_runs,axis=0)
    dccae_ccor_rho_trials[r] = dccae_ccor_runs
    non_mult_ccor_rho[r] = np.mean(non_mult_ccor_runs,axis=0)
    non_mult_ccor_rho_trials[r] = non_mult_ccor_runs
    proposed_ccor_rho[r] = np.mean(proposed_ccor_runs,axis=0)
    proposed_ccor_rho_trials[r] = proposed_ccor_runs
    proposed_dccae_ccor_rho[r] = np.mean(proposed_dccae_ccor_runs,axis=0)
    proposed_dccae_ccor_rho_trials[r] = proposed_dccae_ccor_runs
    
    cca_sub_sim_rho[r] = np.mean(cca_sub_sim_runs,axis=0)
    cca_sub_sim_rho_trials[r] = cca_sub_sim_runs
    dcca_sub_sim_rho[r] = np.mean(dcca_sub_sim_runs,axis=0)
    dcca_sub_sim_rho_trials[r] = dcca_sub_sim_runs
    dccae_sub_sim_rho[r] = np.mean(dccae_sub_sim_runs,axis=0)
    dccae_sub_sim_rho_trials[r] = dccae_sub_sim_runs
    non_mult_sub_sim_rho[r] = np.mean(non_mult_sub_sim_runs,axis=0)
    non_mult_sub_sim_rho_trials[r] = non_mult_sub_sim_runs
    proposed_sub_sim_rho[r] = np.mean(proposed_sub_sim_runs,axis=0)
    proposed_sub_sim_rho_trials[r] = proposed_sub_sim_runs
    proposed_dccae_sub_sim_rho[r] = np.mean(proposed_dccae_sub_sim_runs,axis=0)
    proposed_dccae_sub_sim_rho_trials[r] = proposed_dccae_sub_sim_runs
    
    print('rho value =',rho)
    print('CCA = ',np.mean(cca_sub_sim_rho[r]))
    print('DCCA = ',np.mean(dcca_sub_sim_rho[r]))
    print('DCCAE = ',np.mean(dccae_sub_sim_rho[r]))
    print('IDCCAE = ',np.mean(non_mult_sub_sim_rho[r]))
    print('Proposed = ',np.mean(proposed_sub_sim_rho[r]))
    print('Proposed DCCAE = ',np.mean(proposed_dccae_sub_sim_rho[r]))

---